In [2]:
#Importing all necessary libraries.
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split



In [3]:
 # Import the dataset.
 df= pd.read_excel("/content/Folds5x2_pp.xlsx", sheet_name="Sheet1")


In [4]:
# Checking dataset.
df.head()

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90


The following data presented below is not normalized, then we will proceed to use MinMaxScaler from sklearn to normalize the data.

In [5]:
df.describe()

,AT,V,AP,RH,PE
count,9568.000000,9568.000000,9568.000000,9568.000000,9568.000000
mean,19.651231,54.305804,1013.259078,73.308978,454.365009
std,7.452473,12.707893,5.938784,14.600269,17.066995
min,1.810000,25.360000,992.890000,25.560000,420.260000
25%,13.510000,41.740000,1009.100000,63.327500,439.750000
50%,20.345000,52.080000,1012.940000,74.975000,451.550000
75%,25.720000,66.540000,1017.260000,84.830000,468.430000
max,37.110000,81.560000,1033.300000,100.160000,495.760000


In [6]:
X,Y = df[['AT','V', 'AP', 'RH']], df['PE']
scaler= MinMaxScaler()
X_new= scaler.fit_transform(X)
target_scaler= MinMaxScaler()
Y_new= target_scaler.fit_transform(Y.values.reshape(-1,1))
X_train, X_test, Y_train, Y_test = \
train_test_split(X_new, Y_new, test_size=0.4, random_state=333)

Setting up a class linear regression model.

The following definition function method,  models the function  y= F(X,W); the "fit" method performs the auto gradient and updates the weights and bias, the "predict" method is used to get the output "y" for a given input X , and the "get_weights" method returns the learned weights and bias.

In [36]:
class LinearRegressor(tf.keras.Model):
    def __init__(self, d, lr=0.001):
        super(LinearRegressor, self).__init__()

        # Variables for weight and bias
        self.b = tf.Variable(0.0, dtype=tf.float32)
        self.W = tf.Variable(tf.random.normal([d, 1]), dtype=tf.float32)

        # Stochastic Gradient Descent (SGD) with learning rate to minimize loss
        self.optimizer = tf.optimizers.SGD(lr)

# This part is the model architecture:
    def call(self, inputs):
        # The Linear Regression Model
        return tf.matmul(inputs, self.W) + self.b

    def compute_loss(self, predictions, targets):
        # Loss Function
        return tf.reduce_mean(tf.square(targets - predictions))

    def train_step(self, features, targets):
        with tf.GradientTape() as tape:
            predictions = self(features, training=True)
            loss = self.compute_loss(predictions, targets)

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        return loss

    def fit(self, X, Y, epochs=500):
        total = []
        for epoch in range(epochs):
            loss = self.train_step(X, Y)
            total.append(loss)
            if epoch % 100 == 0:
                print('Epoch {0}/{1} : Loss {2}'.format(epoch, epochs, loss))
        return total

    def predict(self, X):
        return self(X, training=False)

    def get_weight(self):
        return self.W.numpy(), self.b.numpy()

# Example usage:
N, d = X_train.shape
model = LinearRegressor(d)
loss = model.fit(X_train, Y_train, epochs=2000)


Epoch 0/2000 : Loss 0.20339155197143555
Epoch 100/2000 : Loss 0.185870960354805
Epoch 200/2000 : Loss 0.1777181625366211
Epoch 300/2000 : Loss 0.1734531819820404
Epoch 400/2000 : Loss 0.1708071231842041
Epoch 500/2000 : Loss 0.1688406765460968
Epoch 600/2000 : Loss 0.1671648472547531
Epoch 700/2000 : Loss 0.16561855375766754
Epoch 800/2000 : Loss 0.1641349345445633
Epoch 900/2000 : Loss 0.162686288356781
Epoch 1000/2000 : Loss 0.16126087307929993
Epoch 1100/2000 : Loss 0.15985411405563354
Epoch 1200/2000 : Loss 0.1584630310535431
Epoch 1300/2000 : Loss 0.15708748996257782
Epoch 1400/2000 : Loss 0.15572687983512878
Epoch 1500/2000 : Loss 0.1543807089328766
Epoch 1600/2000 : Loss 0.15304897725582123
Epoch 1700/2000 : Loss 0.1517312228679657
Epoch 1800/2000 : Loss 0.15042732656002045
Epoch 1900/2000 : Loss 0.14913734793663025
